In [1]:
#load images
cluster_result_path = '../../datas/cluster_results/kmedoids_pam_200_test_1010.json'
dir_id = 'kmedoids_pam_200_test_1010'
silhouettes_path = f'../../datas/silhouettes/{dir_id}_cleaned'
output_dir = f'../../datas/output_assets/{dir_id}/'
#load preprocessed data\
import sys
sys.path.append('../../src')
from PoseFormatUtils import read_formatted_csv
from clustering.Preprocessor import Preprocessor
from clustering.Distances import distance
import os
import pandas as pd
from tqdm import tqdm

formatted_df = read_formatted_csv('../../datas/filtered_datas/vitpose_filtered_0927.csv')

preprocessor = Preprocessor(formatted_df)
preprocessed_data = preprocessor.preprocess(existance_threshhold = 0.5)



Reading dataframe: ../../datas/filtered_datas/vitpose_filtered_0927.csv...Done!


27462it [00:00, 32389.37it/s]


In [2]:
def get_file_names_in_dir(dir_path):
    file_names = os.listdir(dir_path)
    file_names = [str(file_name) for file_name in file_names if os.path.isfile(os.path.join(dir_path, file_name))]
    file_names.remove('image_datas.csv')
    center_images = [name.split('_',1)[1] for name in file_names]
    center_images = [name.split('.')[0] + '.jpg' for name in center_images]
    return center_images

def get_repr(name):
    idx = formatted_df.loc[formatted_df['name'] == name].index[0]
    return preprocessed_data[idx]


center_image_names = get_file_names_in_dir(silhouettes_path)
centroids = [get_repr(name) for name in center_image_names]


In [3]:
bgcluster_df = pd.read_csv('../../datas/vendors/background_50_0929.csv', index_col=False)
bg_cluster_count = bgcluster_df['label'].max() + 1

#cluster_data[name] = (poseid, bgid)
cluster_data = {}
#pose_spread[poseid] = [bg1count, bg2count...]
pose_spread = [[0 for i in range(bg_cluster_count)] for j in range(len(centroids))]

for index, row in tqdm(bgcluster_df.iterrows()):
    repr = get_repr(row['name'])
    poseID = 0
    min_dist = -1
    for centroidID, centroid in enumerate(centroids):
        dist = distance(centroid, repr)
        if min_dist == -1 or  dist < min_dist:
            poseID = centroidID
            min_dist = dist
    
    bgID = row['label']
    cluster_data[row['name']] = (poseID, bgID)
    pose_spread[poseID][bgID] += 1


0it [00:00, ?it/s]

27462it [12:06, 37.81it/s]


In [4]:
#make rank.csv
rates = [[] for id in range(bg_cluster_count)]
for poseID, spread in enumerate(pose_spread):
    for bgID, c in enumerate(spread):
        rate = c / sum(spread)
        rates[bgID].append((poseID, rate))
    
for bgID in range(len(rates)):
    rates[bgID].sort(key=lambda x : -x[1])

for i in range(bg_cluster_count):
    print(rates[i])

ranks = [[] for i in range(bg_cluster_count)]
for bgid in range(bg_cluster_count):
    ranks[bgid] = [item[0] for item in rates[bgid]]

print(ranks)

[(59, 0.08333333333333333), (15, 0.07547169811320754), (199, 0.07407407407407407), (96, 0.05421686746987952), (12, 0.05384615384615385), (89, 0.0528169014084507), (164, 0.05128205128205128), (57, 0.04918032786885246), (94, 0.048), (68, 0.04568527918781726), (154, 0.045454545454545456), (132, 0.043689320388349516), (147, 0.0425531914893617), (31, 0.041666666666666664), (101, 0.04090909090909091), (51, 0.040345821325648415), (144, 0.04), (106, 0.039735099337748346), (1, 0.0392156862745098), (18, 0.038461538461538464), (197, 0.0380952380952381), (80, 0.03761755485893417), (21, 0.0371900826446281), (23, 0.037037037037037035), (75, 0.037037037037037035), (191, 0.03571428571428571), (0, 0.03529411764705882), (83, 0.03508771929824561), (173, 0.03349282296650718), (24, 0.03333333333333333), (72, 0.033112582781456956), (86, 0.03255813953488372), (62, 0.03164556962025317), (32, 0.031007751937984496), (108, 0.030927835051546393), (43, 0.03089887640449438), (6, 0.030303030303030304), (11, 0.030303

In [30]:
maxval = max(rates, key = lambda x:x[0][1] if x[0][1] < 1.0 else 0)
print(maxval)
print(rates.index(maxval))

[(114, 0.2), (95, 0.08620689655172414), (174, 0.08275862068965517), (155, 0.0821917808219178), (164, 0.07692307692307693), (179, 0.07058823529411765), (121, 0.06976744186046512), (112, 0.06944444444444445), (100, 0.06666666666666667), (187, 0.06521739130434782), (120, 0.06363636363636363), (96, 0.060240963855421686), (13, 0.05982905982905983), (142, 0.058823529411764705), (192, 0.05844155844155844), (18, 0.057692307692307696), (146, 0.05726872246696035), (172, 0.053691275167785234), (65, 0.05263157894736842), (124, 0.05263157894736842), (126, 0.05263157894736842), (163, 0.05185185185185185), (137, 0.05172413793103448), (116, 0.04888888888888889), (94, 0.048), (175, 0.047619047619047616), (153, 0.04519774011299435), (184, 0.04363636363636364), (160, 0.04310344827586207), (156, 0.04294478527607362), (89, 0.04225352112676056), (88, 0.041916167664670656), (23, 0.0411522633744856), (84, 0.0410958904109589), (150, 0.040268456375838924), (144, 0.04), (61, 0.03875968992248062), (173, 0.0382775

In [10]:
rank_data = [{'bg_label':id, 'pose_ids': ranks[id]} for id in range(bg_cluster_count)]

pd.DataFrame(rank_data).to_csv(output_dir + 'rank.csv', index = False)

In [17]:
import cv2
import vis.cv2utils as cv2utils
if not os.path.exists(f'{output_dir}silhouettes'):
    # If it doesn't exist, create the directory
    os.makedirs(f'{output_dir}silhouettes')

def get_png_names_in_dir(dir_path):
    file_names = os.listdir(dir_path)
    file_names = [str(file_name) for file_name in file_names if os.path.isfile(os.path.join(dir_path, file_name))]
    file_names.remove('image_datas.csv')
    return file_names

centroid_images = get_png_names_in_dir(silhouettes_path)

image_datas = []
for id, file_name in tqdm(enumerate(centroid_images)):
    file_name = file_name[:-4] + '.png'
    silhouette = cv2.imread(f'{silhouettes_path}/{file_name}')

    totalwidth, totalheight = silhouette.shape[1], silhouette.shape[0]
    #바운딩박스 구하고, 실루엣을 바운딩박스에 맞게 자르기
    minx, miny, maxx, maxy = totalwidth, totalheight, 0, 0
    for y in range(totalheight):
        for x in range(totalwidth):
            pixel = silhouette[y,x]
            if pixel[0] == 0:
                continue
            minx = min(minx, x)
            miny = min(miny, y)
            maxx = max(maxx, x)
            maxy = max(maxy, y)

    sliced_silhouette = silhouette[miny:maxy+1, minx:maxx+1]
    
    bbwidth, bbheight = maxx - minx, maxy - miny
    center_pos = (minx + bbwidth/2, miny + bbheight/2)
    size = (bbwidth, bbheight)
    image_datas.append({'id':i, 'name':file_name,
                        'center':[center_pos[0] / totalwidth, center_pos[1] / totalheight],
                        'size':[size[0] / totalwidth, size[1] / totalheight]})
    #바운딩박스 center의 위치를 0~1 구해서 넣기, 크기도 0~1 구해서 넣기
    if sliced_silhouette.shape[0] == 0 or sliced_silhouette.shape[1] == 0:
        sliced_silhouette = cv2utils.black_image((256,256))
    cv2.imwrite(f'{output_dir}silhouettes/{id}.png', sliced_silhouette)
pd.DataFrame(image_datas).to_csv(f'{output_dir}image_datas.csv', index=False)

200it [01:33,  2.15it/s]
